In [1]:
from sklearn.decomposition import PCA
from sklearn.cluster import SpectralClustering
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# Simulando dados de usuários e itens
np.random.seed(0)
data = np.random.randint(0, 2, (100, 10))  # 100 usuários, 10 itens

# DataFrame para manipulação dos dados
df = pd.DataFrame(data, columns=[f'Item_{i+1}' for i in range(10)])

# Pipeline para redução de dimensionalidade e clustering
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Normalização dos dados
    ('pca', PCA(n_components=2)),  # Redução de dimensionalidade com PCA
    ('clustering', SpectralClustering(n_clusters=3, affinity='nearest_neighbors', assign_labels='kmeans'))
])

# Aplicando o pipeline para identificar clusters de usuários
clusters = pipeline.fit_predict(df)

# Adicionando o cluster aos dados originais
df['Cluster'] = clusters

# Criando uma nova coluna para consumo acima de 70%
threshold = 0.7 * df.drop('Cluster', axis=1).max(axis=1)
df['High_Consumption'] = (df.drop(['Cluster', 'High_Consumption'], axis=1, errors='ignore').sum(axis=1) > threshold).astype(int)

# Preparando dados para o modelo de classificação
X = df.drop('High_Consumption', axis=1)  # Características
y = df['High_Consumption']  # Variável alvo

# RandomForest para classificar os itens como recomendáveis ou não
rf_pipeline = Pipeline([
    ('random_forest', RandomForestClassifier(n_estimators=100, random_state=0))
])

# Treinando o modelo RandomForest
rf_pipeline.fit(X, y)

# Visualizando os resultados
df['Recommended'] = rf_pipeline.predict(X)
df.head()

,Item_1,Item_2,Item_3,Item_4,Item_5,Item_6,Item_7,Item_8,Item_9,Item_10,Cluster,High_Consumption,Recommended
0,0,1,1,0,1,1,1,1,1,1,0,1,1
1,1,0,0,1,0,0,0,0,0,1,1,1,1
2,0,1,1,0,0,1,1,1,1,0,0,1,1
3,1,0,1,0,1,1,0,1,1,0,0,1,1
4,0,1,0,1,1,1,1,1,0,1,2,1,1
